In [3]:
import numpy as np
import math
def volume_values(a,b,c,alpha, beta, gamma):
    G = np.linalg.det(np.array([[a**2,a*b*np.cos(gamma),a*c*np.cos(beta)],[a*b*np.cos(gamma),b**2,b*c*np.cos(alpha)],[a*c*np.cos(beta),b*c*np.cos(alpha),c**2]]))
    return np.sqrt(G)

def volume_vectors(a1,a2,a3):
    b=[]
    for i in np.cross(a2,a3): b.append(i)
    return abs(np.dot(a1, b))

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('example_cartesian.csv', delim_whitespace=True)

coords = []

for i in range(len(df)):
    coords.append((df['x'][i], df['y'][i], df['z'][i]))


def transform_coordinates(matrix, points):    
    transformed_points = []
    for j in range(len(points)):
        point = []
        # Apply the transformation
        for i in range(3):
            a=[]
            t=0
            for element in matrix[i]:
                a.append(element*points[j][t])
                t=t+1
            a = sum(a)
            point.append(a)
        transformed_points.append(point)
    return transformed_points

# Inverse of A^-1
transformation_matrix = np.array([
    [1/3.08,    1/(2*1.54*np.sqrt(3)),   0],  # Shift x by 2
    [0,         1/(1.54*np.sqrt(3)),     0],  # Shift y by 3
    [0,         0,                       1/10.07]  # Shift z by 4
])

In [4]:
print(volume_values(3.08, 3.08, 10.07, np.pi/2, np.pi/2, np.pi*2/3))
print("For 72 atoms the following vectors give the volume (in A):")
print(volume_vectors((1.54, -2.667358243656071, 0.0), (1.54, 2.667358243656071, 0.0), (0.0, 0.0,10.07)))
a_vec = [1.54, -2.667358243656071, 0.0]
b_vec = [1.54, 2.667358243656071, 0.0]
c_vec = [0.0, 0.0,10.07]
a = np.sqrt(a_vec[0]**2 + a_vec[1]**2 + a_vec[2]**2)
b = np.sqrt(b_vec[0]**2 + b_vec[1]**2 + b_vec[2]**2)
c = np.sqrt(c_vec[0]**2 + c_vec[1]**2 + c_vec[2]**2)
alpha = math.acos(np.dot(b_vec, c_vec)/(b*c))
beta = math.acos(np.dot(a_vec, c_vec)/(a*c))
gamma = math.acos(np.dot(a_vec, b_vec)/(a*b))

print("The parameters (in A) and the volume calculated with them:")
print(a,b,c,alpha*180/np.pi, beta*180/np.pi, gamma*180/np.pi)
print(volume_values(a,b,c,alpha,beta,gamma))
conv = 1.8897259886
print("Now the parameters in bohr:")
print(a*conv,b*conv,c*conv,alpha*180/np.pi, beta*180/np.pi, gamma*180/np.pi)
print(df)

82.72971634193928
For 72 atoms the following vectors give the volume (in A):
82.72971634193924
The parameters (in A) and the volume calculated with them:
3.08 3.08 10.07 90.0 90.0 120.00000000000001
82.72971634193921
Now the parameters in bohr:
5.820356044888 5.820356044888 19.029540705202 90.0 90.0 120.00000000000001
   El             x             y         z
0  Si  0.000000e+00  0.000000e+00 -3.142907
1  Si  0.000000e+00  0.000000e+00  1.892093
2   C  0.000000e+00  0.000000e+00 -5.030368
3  Cr -1.880015e-15 -1.292510e-15  0.004632
4  Si  0.000000e+00 -1.778239e+00 -0.625931
5  Si  0.000000e+00  1.778239e+00  4.409069
6   C  0.000000e+00 -1.778239e+00 -2.519484
7   C  0.000000e+00  1.778239e+00  2.515516


In [5]:
df2 = df.applymap(lambda x: x * 1.8897259886 if isinstance(x, (int, float)) else x)
print(df2)

   El             x             y         z
0  Si  0.000000e+00  0.000000e+00 -5.939234
1  Si  0.000000e+00  0.000000e+00  3.575537
2   C  0.000000e+00  0.000000e+00 -9.506017
3  Cr -3.552713e-15 -2.442490e-15  0.008754
4  Si  0.000000e+00 -3.360384e+00 -1.182838
5  Si  0.000000e+00  3.360384e+00  8.331932
6   C  0.000000e+00 -3.360384e+00 -4.761134
7   C  0.000000e+00  3.360384e+00  4.753636


C:\Users\Mikel\AppData\Local\Temp\ipykernel_41736\636397403.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df.applymap(lambda x: x * 1.8897259886 if isinstance(x, (int, float)) else x)


In [16]:
df2.to_csv('BOHR_unit_cell_Cr_carterian_coords.csv', sep='\t', index=False)

In [6]:
conv = 1.8897259886
coords_bohr = []
for coord in coords:
    a = []
    for i in coord:
        a.append(i*conv)
    coords_bohr.append(a)

#Transformation matrix in bohr
transformation_matrix_bohr = np.array([
    [1/(3.08*conv),    1/(conv*2*1.54*np.sqrt(3)),   0],  # Shift x by 2
    [0,         1/(conv*1.54*np.sqrt(3)),     0],  # Shift y by 3
    [0,         0,                       1/(10.07*conv)]  # Shift z by 4
])

transformed_points = transform_coordinates(transformation_matrix_bohr, coords_bohr)
print(f"Transformed points are: {transformed_points}")
added_transformed_points = []

t=0
for element in df['El']:
    if element == 'Si':
        added_transformed_points.append(np.append(transformed_points[t], +4))
    else:
        added_transformed_points.append(np.append(transformed_points[t], -4))
    t=t+1
added_transformed_points = np.array(added_transformed_points)
print(f"Now this coordinates are transformed from bohr to fractional")
print(added_transformed_points)

Transformed points are: [[0.0, 0.0, -0.31210600000000005], [0.0, 0.0, 0.187894], [0.0, 0.0, -0.49954], [-8.526773824493952e-16, -4.845657289775855e-16, 0.0004599999999995234], [-0.3333333333333335, -0.666666666666667, -0.062158000000000165], [0.3333333333333335, 0.666666666666667, 0.4378419999999999], [-0.3333333333333335, -0.666666666666667, -0.2501969999999999], [0.3333333333333335, 0.666666666666667, 0.24980300000000003]]
Now this coordinates are transformed from bohr to fractional
[[ 0.00000000e+00  0.00000000e+00 -3.12106000e-01  4.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.87894000e-01  4.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.99540000e-01 -4.00000000e+00]
 [-8.52677382e-16 -4.84565729e-16  4.60000000e-04 -4.00000000e+00]
 [-3.33333333e-01 -6.66666667e-01 -6.21580000e-02  4.00000000e+00]
 [ 3.33333333e-01  6.66666667e-01  4.37842000e-01  4.00000000e+00]
 [-3.33333333e-01 -6.66666667e-01 -2.50197000e-01 -4.00000000e+00]
 [ 3.33333333e-01  6.66666667e-01  2.4980